### Sweep Volume, show spectrograms

In [ ]:
# Set parameter file and model to load
from utils.myUtils import mostRecent 

# chose an existing pickledParamFile and run loadModelForEvaluation
pickledParamFile = mostRecent("demoModel/*modelParams.pkl")
#pickledParamFile = mostRecent("/home/lonce/ZCODE/PYTORCH/RNNControl/model/*modelParams.pkl")

# choose saved model to run (defaults to the last saved if loadmodelfile is undefined or None)
# Note: looks in same directory as pickledParamFile
loadModelFileName=None
#loadModelFileName='2018-11-23_09-37-53_model_epoch1_step90.pth'

In [ ]:
# run notebook to load analysis/synthesis code
%run loadModelForEvaluation.ipynb

if 'sample_rate' not in params :   #older versions didn't save sample_rate
    params['sample_rate']=16000

### Priming signal

In [ ]:
# Make noisy nuetral Priming signal 
myp=np.zeros([1,seqLen,len(params['props'])+1])
# !! PRIMING DOES MATTER - has to be a certain level of noise
myp[0,:,0]=.3*np.random.ranf([seqLen])-.15 #signal
myp[0,:,1]= .45+.1*np.random.ranf([seqLen])     #instrument
myp[0,:,2]= .5*np.random.ranf([seqLen])    #pressure1
#myp[0,:,3]=1-myp[0,:,2]    #pressure2
myp[0,:,3]= np.random.ranf([seqLen])    #pitch
myprimer=torch.tensor(myp, dtype=torch.float)

### Volume sweep and visualize

In [ ]:
from IPython.display import Audio, display
import librosa.display

genlen= int(3*params['sample_rate'])
relPNum=7
iNum=1

def fsweep(snum) : 
    t=snum/genlen #t goes from 0 to 1 over the signal duration 
    v= .05 + 1*(.5-np.abs(.5-t))
    p=midi.param2norm(params['lowNote'] + relPNum)
    i=iNum
    return [i,v,p]   

#result=evaluate(genlen, fsweep, chooseTemperature, primeWith=k_primeSignal) #choose a middle value of the param for evaluation
#y=result[k_primeLen:, 0]

y, hdn=generate(rnn, genlen, primer=myprimer, paramvect=fsweep, returnHiddenSequence=True)

y=y[seqLen:]
hdn=hdn[seqLen:]

sr=params['sample_rate']
plt.figure(figsize=(24, 16))
D = librosa.amplitude_to_db(librosa.stft(y, hop_length=128), ref=np.max)
plt.subplot(4, 2, 1)
librosa.display.specshow(D, y_axis='log', sr=sr)
plt.colorbar(format='%+2.0f dB')
plt.title('log-frequency power spectrogram, i= ' + str(iNum) + ", p= " + str(relPNum))


#save wave file
outfilename=k_rootoutname+"/volumeSweep" + "_iNum." + str(iNum) + "_pNum." + str(relPNum) 
librosa.output.write_wav(outfilename + ".wav", y, params['sample_rate'])
plt.savefig(outfilename +".png")

# Sonify signal from last call to generate()
display(Audio(y, rate=params['sample_rate'], autoplay=False))

In [ ]:
# Activation over time for signal from last call to generate()

import matplotlib as mpl
from ipywidgets import interact,interactive
from ipywidgets import FloatSlider, IntSlider
from IPython.display import Audio


#Separate layers for visualization 
#want each layers[n] to be 40 rows of length timesteps=400 (40 x 400)
layers = [np.transpose([x[n] for x in hdn]) for n in range(0,4) ]

t=list(range(0,len(hdn)))
elmts=[list(range(0,len(layers[n][0]))) for n in range(0,4)]

print("Layer[0] has {} rows of length {}".format(len(layers[0]), len(layers[0][0])))

#*************************

def widgetfunc(layer,snum,scale):

    global t
    global layers
    
    print("length of time array = {}".format(len(t)))
    
    endSnum=int(min(snum+40*math.pow(2,scale) , len(t)-1))
    print("endSnum is {}".format(endSnum))
    
    numxticks=10
    xtick_spacing=max(int((endSnum-snum)/numxticks), 1)
    xlabels=range(snum, endSnum, xtick_spacing)
    
    aspectRatio=(endSnum-snum)/40

    
    numplots=40
    f, axarr = plt.subplots(numplots, sharex=True, sharey=True, figsize=(30,30))
    for i in range(numplots) :
        axarr[i].plot(t[snum:endSnum], layers[layer][i,snum:endSnum])
        axarr.flat[i].set(xlabel='samples', ylabel=i)
        axarr[i].set_yticks([])

    
    axarr[0].set_title('Activations for layer {}'.format(layer))

    # Fine-tune figure; make subplots close to each other and hide x ticks for
    # all but bottom plot.
    f.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)
    

    
layerSlider=IntSlider(min=0, max=3, step=1, value=0, continuous_update=False, description='layer')
snumSlider=IntSlider(min=0, max=len(t), step=1, value=0,  continuous_update=False, description='sampleNumber')
scaleSlider=FloatSlider(min=0, max=12, step=.1, value=12,  continuous_update=False, description='scale')

w = interactive(widgetfunc, layer=layerSlider, snum=snumSlider, scale=scaleSlider)
display(w)

### Sonify individual activations 

In [ ]:
# Sonify activation of individual cells over time for instrument visualized above

def widgetfunc(layer,unit,snum,scale):
    global t
    global layers
    
    endSnum=int(min(snum+40*math.pow(2,scale) , len(t)-1))

    display(Audio(layers[layer][unit,snum:endSnum], rate=sr, autoplay=True))
    
layerSlider=IntSlider(min=0, max=3, step=1, value=0, continuous_update=False, description='layer')
unitSlider=IntSlider(min=0, max=40, step=1, value=0, continuous_update=False, description='unit')
snumSlider=IntSlider(min=0, max=len(t), step=1, value=0,  continuous_update=False, description='sampleNumber')
scaleSlider=FloatSlider(min=0, max=12, step=.1, value=12,  continuous_update=False, description='scale')

w = interactive(widgetfunc, layer=layerSlider, unit=unitSlider, snum=snumSlider, scale=scaleSlider)
display(w)
